# I) Installations et imports

In [1]:
# login : eleveenmi@gmail.com  (i=1,2,3,..)
# mwd : pwdeleveEnmi (i=1,2,3,..)

# Google sheet
!pip install --upgrade gspread gspread_dataframe

from google.colab import auth
from google.auth import default
import gspread
from google.colab import drive


KeyboardInterrupt: 

# II) Authentification

In [ ]:
# pour colab :
auth.authenticate_user()
# pour accéder aux fichiers google (google sheet)
creds, _ = default()  # or use service account
gc = gspread.authorize(creds)

#
drive.mount('/content/drive')

# III) Récupération des images :

# IV) Récupération d'une partie des labels :

In [ ]:
# Open by ID or URL
sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/1KT4RXpX3FkR2APlGXhD54euEpPWHbFZKrxoWdFzeqog")
worksheet = sh.sheet1
data = worksheet.get_all_records()

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df.head()

,Image,Classe,Nebul,Hauteur
0,202209071633_S.jpg,41,4,2
1,202107301651_S.jpg,,,
2,202108161555_S.jpg,,,
3,202012042334_S.jpg,,,


# V) Annotation